In [1]:
import dnnlib
import legacy
import torch
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import scipy.interpolate
import math
import random
import preprocessing.image_utils as itools
import cv2
import synthesis.synthesis as synth
from tqdm import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
network_pkl = "../t-golden-fid14.19.pkl"
device = torch.device('cuda')
G = synth.load_generator(network_pkl, device)

Loading networks from "../t-golden-fid14.19.pkl"...
Network loaded.


In [4]:
synth.generate_videos(10, "../outputs/videos/birthday", G, device, size=(3,3), duration=30)

Seed count:  135.0


100%|██████████████████████████████████████████| 3600/3600 [23:12<00:00,  2.58it/s]


Seed count:  135.0


100%|██████████████████████████████████████████| 3600/3600 [23:12<00:00,  2.59it/s]


Seed count:  135.0


...

Seed count:  135.0


100%|██████████████████████████████████████████| 3600/3600 [23:13<00:00,  2.58it/s]


Seed count:  135.0


 13%|█████▍                                     | 452/3600 [02:54<20:16,  2.59it/s]


KeyboardInterrupt: 